# 📗 *EcoNormasBot: Consultas Inteligentes sobre el Pacto Verde Europeo con IA Generativa*

## ✨ Descripción General

Este notebook corresponde al proyecto final del **Intensive Generative AI Capstone de 5 días**. En él se muestra cómo implementar un sistema de generación aumentada por recuperación (RAG) para facilitar la consulta de múltiples documentos oficiales de la Unión Europea relacionados con el Pacto Verde Europeo. El sistema integra **Google Gemini** para generación de texto y **FAISS** para búsqueda semántica eficiente.

**Caso de uso:** Permitir a las personas interactuar con políticas públicas complejas de la UE (como reglamentos, planes estratégicos o informes de seguimiento) mediante preguntas formuladas en lenguaje natural.

---

## ✅ Capacidades de IA Generativa Aplicadas

- **Embeddings semánticos**: generación de vectores representativos de fragmentos documentales utilizando la API de Gemini.
- **Búsqueda Vectorial (FAISS)**: identificación rápida y precisa de contenidos relevantes a través de comparación de vectores.
- **Generación Aumentada por Recuperación (RAG)**: uso de fragmentos recuperados como contexto para la generación de respuestas fundamentadas por un LLM.
- **Comprensión de Documentos Normativos**: respuestas contextuales basadas en documentos de políticas públicas reales.

---

## 📄 Documentos de Referencia

El sistema opera con tres documentos oficiales y accesibles públicamente de la Unión Europea:

1. **Comunicación sobre el Pacto Verde Europeo (2019)**  
2. **Plan Industrial del Pacto Verde (2023)** – COM(2023) 62 final  
3. **Informe de seguimiento del Pacto Verde (2023)** – JRC140372  

Cada documento es procesado, dividido en fragmentos y acompañado de metadatos relevantes (título y número de página).

---

## 🚀 Funcionamiento del Sistema

1. Se cargan los archivos PDF y se fragmentan en secciones con superposición.
2. Cada fragmento se transforma en un embedding mediante la API de Gemini.
3. Los vectores resultantes se indexan utilizando FAISS.
4. Ante una consulta del usuario:
   - El sistema recupera los fragmentos más pertinentes.
   - Gemini Pro genera una respuesta precisa fundamentada en dichos fragmentos.
   - La respuesta final incluye las **fuentes** (documento y página correspondiente).

---

ℹ️ En la sección 5 encontrarás ejemplos de preguntas que puedes realizar al sistema.

➡️ ¡Pasemos al código!


## 📥 1. Cargar Documentos PDF

En esta sección se cargan tres documentos oficiales de la Unión Europea relacionados con el Pacto Verde Europeo, en formato PDF.  
Utilizamos la biblioteca `PyMuPDF` para extraer el texto página por página, conservando tanto el título del documento como el número de página para referencias posteriores.

Este paso prepara los datos para las siguientes etapas de fragmentación (chunking) e indexación semántica.

In [1]:
# -------------------------
# 1. Cargar Documentos PDF
# -------------------------

# Instalamos la biblioteca necesaria para trabajar con archivos PDF
!pip install -q pymupdf

import fitz  # PyMuPDF

# Función para extraer el texto de un PDF, página por página
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    return [page.get_text() for page in doc]

# Rutas a los documentos PDF oficiales cargados en el entorno
pdf_paths = {
    "Pacto Verde 2019": "/kaggle/input/europeangreendealdocs/cellar_b828d165-1c22-11ea-8c1f-01aa75ed71a1.0002.02_DOC_1.pdf",
    "Plan Industrial 2023": "/kaggle/input/europeangreendealdocs/CELEX_52023DC0062_EN_TXT.pdf",
    "Informe de Progreso 2023": "/kaggle/input/europeangreendealdocs/JRC140372_01.pdf"
}


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 83.4 MB/s eta 0:00:00


## ✂️ 2. Fragmentar el Texto Combinado

Dividimos cada documento en fragmentos de texto con superposición para preservar el contexto y mejorar la calidad de los embeddings.  
Cada fragmento se etiqueta con su fuente original (nombre del documento y número de página), lo que permite citar adecuadamente el origen de cualquier respuesta generada más adelante.


In [2]:
# -------------------------
# 2. Fragmentar el Texto Combinado
# -------------------------

# Definimos listas para almacenar los fragmentos de texto y sus fuentes
chunks = []
chunk_sources = []

# Tamaño de cada fragmento (número de caracteres) y solapamiento entre fragmentos
chunk_size = 500
overlap = 100

# Extraemos y fragmentamos el texto de cada documento, asociando la fuente (nombre del documento + página)
for label, path in pdf_paths.items():
    pages = extract_text_from_pdf(path)
    for page_num, text in enumerate(pages):
        start = 0
        while start < len(text):
            end = start + chunk_size
            chunk = text[start:end]
            source = f"{label}, página {page_num + 1}"
            chunks.append((chunk, source))
            start += chunk_size - overlap

# Mostramos el total de fragmentos generados
print(f"Número total de fragmentos generados: {len(chunks)}")



Número total de fragmentos generados: 1699


## 🔗 3. Generar Embeddings con la API de Gemini

Utilizamos el modelo `embedding-001` de Gemini para convertir cada fragmento de texto en una representación vectorial.  
Estos embeddings permiten realizar búsquedas por similitud y localizar posteriormente los pasajes más relevantes en función de la consulta del usuario.


In [3]:
# -------------------------
# 3. Generar Embeddings con la API de Gemini
# -------------------------

# Instalamos la biblioteca oficial de Google Generative AI
!pip install -U -q google-generativeai

import google.generativeai as genai
import numpy as np

# Obtenemos la clave secreta de la API desde el entorno seguro de Kaggle
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GOOGLE_API_KEY")

# Configuramos la API de Gemini con la clave obtenida
genai.configure(api_key=secret_value_0)

# Función que obtiene el embedding de un fragmento de texto utilizando Gemini
def get_google_embedding(text):
    response = genai.embed_content(
        model="models/embedding-001",
        content=text,
        task_type="retrieval_document"
    )
    return response["embedding"]

# Generamos los embeddings para todos los fragmentos
embeddings = []
for i, (chunk, _) in enumerate(chunks):
    try:
        emb = get_google_embedding(chunk)
        embeddings.append(emb)
    except Exception as e:
        # Si ocurre un error, añadimos un vector nulo y lo notificamos
        print(f"Error en el fragmento {i}: {e}")
        embeddings.append([0.0] * 768)




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 5.4 MB/s eta 0:00:00


## 📚 4. Indexar los Fragmentos con FAISS

Utilizamos la biblioteca FAISS (desarrollada por Facebook) para construir un índice vectorial en memoria.  
Este índice permite recuperar de forma rápida y eficiente los fragmentos de texto más relevantes desde el punto de vista semántico, en función de las preguntas formuladas por el usuario.  
La búsqueda de similitud se basa en la distancia L2.

In [4]:
# -------------------------
# 4. Indexar los Fragmentos con FAISS
# -------------------------

# Instalamos la versión de FAISS optimizada para CPU
!pip install -q faiss-cpu

import faiss

# Obtenemos la dimensión de los vectores de embedding generados
embedding_dim = len(embeddings[0])

# Convertimos la lista de embeddings a una matriz NumPy en formato float32 (requerido por FAISS)
embedding_matrix = np.array(embeddings).astype("float32")

# Creamos un índice plano basado en distancia L2 (euclidiana)
index = faiss.IndexFlatL2(embedding_dim)

# Añadimos la matriz de embeddings al índice
index.add(embedding_matrix)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 55.0 MB/s eta 0:00:00


## 💬 5. RAG: Consultar con Gemini

Esta es la fase central del sistema RAG (Generación Aumentada por Recuperación).

Para cada pregunta del usuario:
1. Se genera el embedding de la pregunta.
2. Se consulta el índice FAISS para recuperar los fragmentos más relevantes.
3. Se combina el contexto recuperado con la pregunta y se envía a Gemini Pro para generar una respuesta.
4. Se añaden citas de las fuentes correspondientes utilizando los metadatos de los fragmentos.


In [5]:

# -------------------------
# 5. RAG: Consultar con Gemini
# -------------------------

# Cargamos el modelo generativo más reciente de Gemini 1.5 Pro
rag_model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")

# Función para buscar los fragmentos más similares a una consulta del usuario
def search_similar_chunks(query, top_k=10):
    query_emb = np.array([get_google_embedding(query)], dtype='float32')  # Genera embedding de la consulta
    distances, indices = index.search(query_emb, top_k)  # Busca los fragmentos más cercanos en el índice FAISS
    return [chunks[i] for i in indices[0]]  # Devuelve los fragmentos correspondientes

# Función principal para responder una pregunta con contexto (RAG)
def ask_with_context(query, top_k=10):
    selected = search_similar_chunks(query, top_k)  # Recupera los fragmentos más relevantes
    context = "\n\n".join([chunk for chunk, _ in selected])  # Une los textos para usarlos como contexto
    sources = set([source for _, source in selected])  # Extrae las fuentes para citarlas

    # Prompt para Gemini: respuesta basada estrictamente en el contexto proporcionado
    prompt = f"""
Eres un asistente que responde preguntas sobre el Pacto Verde Europeo, utilizando únicamente documentos oficiales.

Responde de forma objetiva, sin especulación. Cita programas específicos, sectores, fechas o cifras cuando sea posible.

Contexto:
{context}

Pregunta:
{query}

Respuesta:
"""
    # Genera la respuesta con Gemini
    response = rag_model.generate_content(prompt)

    # Añade una nota con las fuentes utilizadas tipo lista
    source_note = "\n\nFuentes:\n" + "\n".join(f"- {src}" for src in sorted(set(sources)))
    return response.text + source_note




## 💬 Escribe tu pregunta o utiliza alguno de los ejemplos a continuación


In [6]:
# Preguntas de ejemplo que puedes probar
sample_questions = [
    "¿Cuáles son los principales objetivos del Pacto Verde Europeo?",
    "¿Qué sectores industriales están priorizados en el Plan Industrial del Pacto Verde?",
    "¿Qué avances se han reportado hasta 2023 en la implementación del Pacto Verde?",
    "¿Qué medidas propone la UE para alcanzar la neutralidad climática?",
    "¿Qué papel juegan las energías renovables en el Pacto Verde Europeo?"
]

# Prueba una de las preguntas
print(ask_with_context(sample_questions[0]))


El Pacto Verde Europeo busca transformar la UE en una economía moderna, eficiente en el uso de los recursos y competitiva que garantice lo siguiente:

* **No haya emisiones netas de gases de efecto invernadero en 2050.**  (Esto está establecido en la [Comunicación de la Comisión al Parlamento Europeo, al Consejo Europeo, al Consejo, al Comité Económico y Social Europeo y al Comité de las Regiones  El Pacto Verde Europeo COM(2019) 640 final](https://eur-lex.europa.eu/legal-content/ES/TXT/?uri=CELEX%3A52019DC0640)).

* **El desacoplamiento del crecimiento económico del uso de los recursos.**

* **Que ninguna persona ni ningún lugar se quede atrás.**

Adicionalmente, el Plan de Acción "Hacia una contaminación cero del aire, el agua y el suelo" [COM(2021) 245 final], un componente clave del Pacto Verde Europeo, establece objetivos clave para 2030 para acelerar la reducción de la contaminación en su origen:

* **Mejorar la calidad del aire para reducir el número de muertes prematuras causad

## 💬 Algunas preguntas adicionales de ejemplo

* ¿Cuáles son los pilares principales del Pacto Verde en la UE?
* ¿Qué sectores han mostrado mayores avances hacia los objetivos del Pacto Verde?
* ¿Qué mecanismos de financiación se proponen en el marco del Pacto Verde Europeo?
